In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
os.chdir('C:\\Users\\nbg05\\Local Documents\\Projects\WhiteHeaven')

from ipywidgets import interact
from tqdm import tqdm

In [5]:
# load the dataset
comparison_scores_dataset = np.load('puzzle/data/comparison_scores.npy')

num_pieces = comparison_scores_dataset.shape[0]

In [ ]:
# sub dataset of sorted comparisons
num_comparisons = 64
# store the 64 best comparisons for each edge
sorted_scores_dataset = np.zeros((num_pieces, 4, num_comparisons), dtype=np.float32)
sorted_pieces_dataset = np.zeros((num_pieces, 4, num_comparisons), dtype=np.int32)
sorted_indices_dataset = np.zeros((num_pieces, 4, num_comparisons), dtype=np.int32)

indices = [(piece, i) for i in range(4) for piece in range(num_pieces)]

for piece, i in tqdm(indices, 'sorting'):
    comparison_scores = comparison_scores_dataset[piece, i, :, :]
    flattened_scores = comparison_scores.flatten()
    sort = np.argsort(flattened_scores)[0:num_comparisons]
    
    sorted_pieces_dataset[piece, i, :], sorted_indices_dataset[piece, i, :] = np.unravel_index(sort, comparison_scores.shape)
    sorted_scores_dataset[piece, i, :] = flattened_scores[sort]

In [ ]:
print(sorted_scores_dataset.shape)

In [ ]:
def zip_sorted(piece, i, end = 64):
    return zip(sorted_scores_dataset[piece, i, :end], sorted_pieces_dataset[piece, i, :end], sorted_indices_dataset[piece, i, :end])

def generate_matches(piece, index, num_edges_to_check = 16):
    matches = []
    next_matches = []
    values = []
    
    edge1 = index
    edge2 = (index + 1) % 4
    
    # check if the side is flat
    if sorted_scores_dataset[piece, edge1, 0] == np.inf:
        return None, None, None
    
    # check if the next side is flat
    if sorted_scores_dataset[piece, edge2, 0] == np.inf:
        end = num_edges_to_check * num_edges_to_check
        return np.stack((sorted_pieces_dataset[piece, edge1, :end], sorted_indices_dataset[piece, edge1, :end]), axis=1), None, sorted_scores_dataset[piece, edge1, :end]
    
    for score1, piece1, i1 in zip_sorted(piece, edge1, num_edges_to_check):
        for score2, piece2, i2 in zip_sorted(piece, edge2, num_edges_to_check):
            
            corner_index1 = (i1 + 3) % 4
            corner_index2 = (i2 + 1) % 4
            
            # find the corner that matches the best
            comparisons1 = comparison_scores_dataset[piece1, corner_index1, :, :]
            comparisons2 = comparison_scores_dataset[piece2, corner_index2, :, :]
            
            # reorder comparisons2 so that the edges are in the same order as in comparisons1
            comparisons1 = np.concatenate((comparisons1[:, 1:], comparisons1[:, :1]), axis=1)
            
            score = score1 + score2 + np.min(comparisons1 + comparisons2)
            values.append(score)
            
            matches.append((piece1, i1))
            next_matches.append((piece2, i2))
    
    # sort the values
    matches = np.array(matches)
    next_matches = np.array(next_matches)
    values = np.array(values)
    
    sort = np.argsort(values)
    return matches[sort], next_matches[sort], values[sort]

def find_matches_recursive(potential_matches, depth=10, 
                           current_matches=[], score=0, next_match=None,
                           best_matches=None, best_score=np.inf):
    """recursively solves the edges around the 3x3 grid of pieces

    Args:
        potential_matches (list): list of potential matches for each side stored as a tuple of (matches, next_match, score)
        depth (int): how many matches to check
    """
    
    if len(current_matches) == 4:
        if score < best_score:
            best_score = score
            best_matches = current_matches
        return best_matches, best_score
    
    matches, next_matches, scores = potential_matches[len(current_matches)]
    
    # check if the match is on an edge
    if matches is None:
        return find_matches_recursive(potential_matches, depth,
                                      current_matches + [None],
                                      score,
                                      None,
                                      best_matches, best_score)
    
    
    for i in range(depth):
        if next_match is not None and not np.array_equal(next_match, matches[i]):
            continue
        
        next_match2 = None
        if next_matches is not None:
            next_match2 = next_matches[i]
        
        best_matches, best_score = find_matches_recursive(potential_matches, depth, 
                                                           current_matches + [matches[i]], 
                                                           score + scores[i], 
                                                           next_match2,
                                                           best_matches, best_score)
    
    return best_matches, best_score

def solve_piece_sides(piece):
    """Solves the sides of the puzzle
    """
    
    # try to solve the 3x3 grid of pieces centered the piece
    
    # try to solve each of the 2x2 corners
    # first try to find the best edges, then check how well the corners match
    # we want to try as many edge combinations as possible
    
    # stores indices of the best edges from the sorted scores dataset
    #best_edges = np.zeros((4), dtype=np.int32)
    
    potential_matches = []
    for i in range(4):
        matches, next_matches, values = generate_matches(piece, i, 10)
        potential_matches.append((matches, next_matches, values))
        
    
    best_matches, best_score = find_matches_recursive(potential_matches=potential_matches, depth=20)
    
    return best_matches, best_score
        

# for i in range(100):
#     values, pieces, edges = solve_corner(i, 0)
#     if pieces[0][0] < 127:
#         continue
#     print(i, values[0], pieces[0])

for i in range(100):
    print(solve_piece_sides(i))

In [6]:
side_indices = [(piece, i) for i in range(4) for piece in range(num_pieces)]

In [85]:
side_comparison_scores = comparison_scores_dataset.copy()

for piece1, i1 in side_indices:
    for piece2, i2 in side_indices:
        max_score = max(side_comparison_scores[piece1, i1, piece2, i2], side_comparison_scores[piece2, i2, piece1, i1])
        side_comparison_scores[piece1, i1, piece2, i2] = max_score
        side_comparison_scores[piece2, i2, piece1, i1] = max_score

In [36]:
# find min of side scores
min_index = np.unravel_index(np.argmin(side_comparison_scores), side_comparison_scores.shape)
print(min_index, side_comparison_scores[min_index])

(521, 3, 307, 2) 0.49642524


In [49]:
def get_match_subscore(piece1, side1, piece2, side2, depth=10, comparison_scores=side_comparison_scores, side_sorts=None):
    """gets half of the score for the match between two sides
    """
    
    # rotate the sides
    side1 = (side1 + 1) % 4
    side2 = (side2 + 3) % 4
    
    scores1 = comparison_scores[piece1, side1, :, :]
    scores2 = comparison_scores[piece2, side2, :, :]
    
    if side_sorts is None:
        sort_scores1 = np.stack(np.unravel_index(np.argsort(scores1.flatten())[:depth], scores1.shape), axis=1)
        sort_scores2 = np.stack(np.unravel_index(np.argsort(scores2.flatten())[:depth], scores2.shape), axis=1)
    else:
        # depth should be baked into the side_sorts
        sort_scores1 = side_sorts[piece1, side1, :depth]
        sort_scores2 = side_sorts[piece2, side2, :depth]
    
    # check for edge pieces
    is_edge1 = scores1[sort_scores1[0, 0], sort_scores1[0, 1]] == np.inf
    is_edge2 = scores2[sort_scores2[0, 0], sort_scores2[0, 1]] == np.inf
    if is_edge1 and is_edge2:
        return 0
    if is_edge1 or is_edge2:
        return np.inf
    
    best_score = np.inf
    for piece11, side11 in sort_scores1:
        for piece22, side22 in sort_scores2:
            score = scores1[piece11, side11] + scores2[piece22, side22]
            side11 = (side11 + 1) % 4
            side22 = (side22 + 3) % 4
            score += side_comparison_scores[piece11, side11, piece22, side22]
            
            if score < best_score:
                best_score = score
    
    return best_score
                
    
    

def get_match_score(piece1, edge1, piece2, edge2, depth=10, comparison_scores=side_comparison_scores, side_sorts=None):
    """uses the 3x2 rectangle of pieces around the edge to get how well an edge matches with another edge
    """
    
    num_comparisons = 1
    
    score = comparison_scores[piece1, edge1, piece2, edge2]
    subscore = get_match_subscore(piece1, edge1, piece2, edge2, depth, comparison_scores, side_sorts)
    if subscore != 0:
        num_comparisons += 3
    score += subscore
    subscore = get_match_subscore(piece2, edge2, piece1, edge1, depth, comparison_scores, side_sorts)
    if subscore != 0:
        num_comparisons += 3
    score += subscore
    
    return score / num_comparisons

get_match_score(307, 2, 521, 3, depth=10)

0.9425724574497768

In [22]:
depth = 10
# speeds up computation drastically
side_sorts = np.zeros((num_pieces, 4, depth, 2), dtype=np.int32)

for piece, side in side_indices:
    scores = side_comparison_scores[piece, side, :, :]
    side_sorts[piece, side] = np.stack(np.unravel_index(np.argsort(scores.flatten())[:depth], scores.shape), axis=1)

In [81]:
def get_compound_comparison_scores(piece, i, end=100, depth=10, comparison_scores=side_comparison_scores, side_sorts=side_sorts):
    """gets the comparison scores for the 3x2 rectangle of pieces around the edge
    """
    
    scores = comparison_scores[piece, i, :, :].copy()
    scores_flat = scores.flatten()
    sort = np.argsort(scores_flat)
    indices = np.stack(np.unravel_index(sort, scores.shape), axis=1)

    compound_scores = np.full((num_pieces, 4), np.inf, dtype=np.float32)

    for piece_, i_ in indices[:end]:
        compound_scores[piece_, i_] = get_match_score(piece, i, piece_, i_, depth=depth, comparison_scores=comparison_scores, side_sorts=side_sorts)
    
    return compound_scores

In [84]:
scores = get_compound_comparison_scores(90, 0, depth=10)

# sort the scores
scores_flat = scores.flatten()
sort = np.argsort(scores_flat)
indices = np.stack(np.unravel_index(sort, scores.shape), axis=1)

for i in range(10):
    print(indices[i], scores_flat[sort[i]])

[0 0] 0.8756811
[46  2] 0.960963
[121   2] 0.96568084
[3 3] 0.9822008
[6 2] 0.9962305
[99  2] 1.0310769
[33  2] 1.0445778
[54  2] 1.0890146
[82  2] 1.1528852
[101   2] 1.2444286


In [91]:
def compute_confidence(scores):
    scores_flat = scores.flatten()
    sort = np.argsort(scores_flat)
    scores_sorted = scores_flat[sort]
    
    if scores_sorted[0] == np.inf:
        return 0
    if scores_sorted[3] == np.inf:
        return 0
    
    return (scores_sorted[1] - scores_sorted[0]) / scores_sorted[0]

In [88]:
depth = 10
# speeds up computation drastically
side_sorts = np.zeros((num_pieces, 4, depth, 2), dtype=np.int32)

for piece, side in tqdm(side_indices, 'sorting'):
    scores = side_comparison_scores[piece, side, :, :]
    side_sorts[piece, side] = np.stack(np.unravel_index(np.argsort(scores.flatten())[:depth], scores.shape), axis=1)

compound_comparisons = np.zeros((num_pieces, 4, num_pieces, 4), dtype=np.float32)
comparisons_confidence = np.zeros((num_pieces, 4), dtype=np.float32)
for piece, i in tqdm(side_indices, 'compound comparisons'):
    compound_comparisons[piece, i] = get_compound_comparison_scores(piece, i, depth=10, comparison_scores=side_comparison_scores, side_sorts=side_sorts)
    comparisons_confidence[piece, i] = compute_confidence(compound_comparisons[piece, i])

compound comparisons: 100%|██████████| 4000/4000 [02:01<00:00, 32.85it/s]


In [96]:
flat = comparisons_confidence.flatten()
sort = np.argsort(flat)[::-1]
indices = np.stack(np.unravel_index(sort, comparisons_confidence.shape), axis=1)
confidence_sorted = flat[sort]

for i in range(100):
    print(indices[i], confidence_sorted[i])

[78  2] 0.876274
[92  0] 0.86847323
[82  0] 0.826007
[103   2] 0.77658665
[2 1] 0.7452577
[123   2] 0.7391938
[2 2] 0.72701347
[88  2] 0.6530909
[47  2] 0.6097018
[118   0] 0.60761553
[102   2] 0.56289256
[11  0] 0.56018376
[100   2] 0.5331982
[20  2] 0.53290594
[98  0] 0.5126133
[22  0] 0.5057962
[174   1] 0.49121135
[89  2] 0.49060485
[714   3] 0.4832786
[79  0] 0.48223656
[3 2] 0.47693506
[84  1] 0.46789846
[117   2] 0.46595997
[20  0] 0.4619983
[114   0] 0.4577585
[411   1] 0.4554604
[39  2] 0.45474157
[580   3] 0.45414373
[98  2] 0.45226398
[1 0] 0.45148742
[7 2] 0.44787574
[24  2] 0.44481486
[80  2] 0.4445685
[123   1] 0.44203466
[688   1] 0.44161656
[649   3] 0.43578652
[95  0] 0.43550187
[26  0] 0.43546438
[74  2] 0.43207163
[38  0] 0.4305693
[438   2] 0.43043393
[239   0] 0.42863774
[857   0] 0.42808807
[86  0] 0.42505312
[226   3] 0.4222312
[107   0] 0.42193645
[679   0] 0.4200698
[43  2] 0.41908813
[611   2] 0.41799822
[119   2] 0.4174783
[619   2] 0.41662443
[582   2] 0.415

# Lets solve the puzzle!!! :D